In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install pyngrok

In [ ]:
import cv2
import numpy as np
import tensorflow as tf

# Preprocess function
def preprocess_image(image_path, target_size=(250, 250)):
    img = cv2.imread(image_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = cv2.resize(img, target_size)
    img = tf.keras.applications.mobilenet_v2.preprocess_input(img)
    img = np.expand_dims(img, axis=0)
    return img

# Prediction function
def predict_image_class(model, image_path, class_labels):
    img = preprocess_image(image_path)
    predictions = model.predict(img)
    predicted_index = np.argmax(predictions, axis=1)[0]
    return class_labels[predicted_index]



In [ ]:
from flask import Flask, request, jsonify
from tensorflow.keras.models import load_model
import os
from pyngrok import ngrok

# Ngrok setup
ngrok.set_auth_token("2usn7XXW3bb2TFF7VjRvhxGyHIV_5gWFx9m7hL9wiu9ToyYhB")
public_url = ngrok.connect(5000).public_url

# Initialize Flask app
app = Flask(__name__)

# Load the trained models
CAT_MODEL_PATH = "/content/drive/MyDrive/Anees AI/Models/cat_model.h5"
DOG_MODEL_PATH = "/content/drive/MyDrive/Anees AI/Models/dog_model.h5"
cat_model = load_model(CAT_MODEL_PATH)
dog_model = load_model(DOG_MODEL_PATH)
print("Models loaded successfully!")

# Define class labels for cats and dogs
CAT_class_labels = [
    'Abyssinian', 'Bengal', 'Birman', 'Bombay',
    'British Shorthair', 'Egyptian Mau', 'Maine Coon',
    'Persian', 'Ragdoll', 'Russian Blue',
    'Siamese', 'Sphynx'
]

DOG_class_labels = [
    'american', 'basset', 'beagle', 'boxer', 'chihuahua',
    'english', 'german', 'great', 'havanese', 'japanese',
    'keeshond', 'leonberger', 'miniature', 'newfoundland',
    'pomeranian', 'pug', 'saint', 'samoyed', 'scottish',
    'shiba', 'staffordshire', 'wheaten', 'yorkshire'
]


@app.route('/Cat', methods=['POST'])
def predict_cat():
    """
    Endpoint to predict the cat breed from an uploaded image.
    Returns:
        JSON response with predicted class and confidence score.
    """
    if 'image' not in request.files:
        return jsonify({"error": "No image file provided."}), 400

    image_file = request.files['image']
    if image_file.filename == '':
        return jsonify({"error": "No image file selected."}), 400

    try:
        # Save the uploaded file temporarily
        temp_path = "temp_image.jpg"
        image_file.save(temp_path)

        # Preprocess the image
        img = preprocess_image(temp_path)

        # Make predictions
        predictions = cat_model.predict(img)
        predicted_index = np.argmax(predictions, axis=1)[0]
        predicted_label = CAT_class_labels[predicted_index]
        confidence = float(np.max(predictions))

        # Cleanup temporary file
        os.remove(temp_path)

        return jsonify({
            "predicted_class": predicted_label,
            "confidence": confidence
        })

    except Exception as e:
        return jsonify({"error": str(e)}), 500

@app.route('/Dog', methods=['POST'])
def predict_dog():
    """
    Endpoint to predict the dog breed from an uploaded image.
    Returns:
        JSON response with predicted class and confidence score.
    """
    if 'image' not in request.files:
        return jsonify({"error": "No image file provided."}), 400

    image_file = request.files['image']
    if image_file.filename == '':
        return jsonify({"error": "No image file selected."}), 400

    try:
        # Save the uploaded file temporarily
        temp_path = "temp_image.jpg"
        image_file.save(temp_path)

        # Preprocess the image
        img = preprocess_image(temp_path)

        # Make predictions
        predictions = dog_model.predict(img)
        predicted_index = np.argmax(predictions, axis=1)[0]
        predicted_label = DOG_class_labels[predicted_index]
        confidence = float(np.max(predictions))

        # Cleanup temporary file
        os.remove(temp_path)

        return jsonify({
            "predicted_class": predicted_label,
            "confidence": confidence
        })

    except Exception as e:
        return jsonify({"error": str(e)}), 500

# Display the public URL for accessing the app
print(f"To access the app globally, click here: {public_url}")

# Run the Flask app
if __name__ == '__main__':
    app.run(port=5000)


Models loaded successfully!
To access the app globally, click here: https://7efc-35-234-36-170.ngrok-free.app
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit


1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step


INFO:werkzeug:127.0.0.1 - - [27/Mar/2025 21:03:36] "POST /Dog HTTP/1.1" 200 -


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 93ms/step


...

INFO:werkzeug:127.0.0.1 - - [27/Mar/2025 21:50:00] "POST /Cat HTTP/1.1" 200 -


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 91ms/step


INFO:werkzeug:127.0.0.1 - - [27/Mar/2025 21:50:17] "POST /Dog HTTP/1.1" 200 -


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 85ms/step


INFO:werkzeug:127.0.0.1 - - [27/Mar/2025 21:50:28] "POST /Dog HTTP/1.1" 200 -
